In [1]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=a5278b1a63dfdd2b753739d53213207fbab16fe4c55a29925e6aab9fe00bb964
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import re
import json
import os
from tqdm import tqdm
from pylatexenc.latex2text import LatexNodes2Text

def compare_ignore_spaces(str1, str2):
    """
    Compares two strings while ignoring spaces between characters.
    Returns True if they are equal when spaces are removed, otherwise False.
    """
    return str1.replace(" ", "") == str2.replace(" ", "")

def extract_final_answer(text, marker="Answer:"):
    """
    Extract the final answer following the specified marker.
    """
    if marker in text:
        return text.split(marker)[-1].strip()
    return None


def latex_to_text(latex_string):
    return LatexNodes2Text().latex_to_text(latex_string)


def extract_boxed_content(text):
    """
    Extracts all content enclosed within curly brackets {} only if it appears in the format \(\boxed{...}\).
    If no boxed content is found, it extracts the content that follows "the answer is:".
    
    :param text: The input string containing content in the boxed format or the phrase "the answer is:".
    :return: The extracted content from either the boxed format or after "the answer is:".
    """
    # Check for boxed content first across the entire text
    if "oxed" in text:
        # Find the start position of the opening curly bracket after "boxed"
        match = re.search(r'oxed\s*\{', text)
        if match:
            start_pos = match.end() - 1  # Position of the opening curly bracket
            count = 1  # Counter for nested brackets
            end_pos = start_pos
            
            # Find the matching closing bracket by counting opened and closed brackets
            for i in range(start_pos + 1, len(text)):
                if text[i] == '{':
                    count += 1
                elif text[i] == '}':
                    count -= 1
                
                if count == 0:
                    end_pos = i
                    break
            
            if end_pos > start_pos:
                boxed_content = text[start_pos + 1:end_pos]
                return boxed_content
    
    # If no boxed content is found, check for "the answer is:" followed by content
    answer_match = re.search(r'the answer is\s*:\s*(.*)', text, re.IGNORECASE)
    if answer_match:
        answer_content = answer_match.group(1).strip()
        return answer_content
    
    return None  
 

def test_model_on_math(model, tokenizer, target_device, num_examples=500, is_correct=False):
    """
    Test the model on the MATH dataset.
    """
    # Force model to use a single device
    model = model.to(target_device)
    
    # Try to load the MATH dataset with error handling
    try:
        dataset = load_dataset("math-ai/math500", split="test")
    except Exception as e:
        print(f"Error loading MATH dataset: {e}")       
    
    
    num_examples = min(num_examples, len(dataset))
    print("num_examples", num_examples)
    
    print(f"Limiting to {num_examples} examples (max available in dataset)")
    
    results = []
    total = 0
    correct = 0
    correct_by_level = {level: {"correct": 0, "total": 0} for level in range(1, 6)}
    correct_by_type = {}
    
    system_prompt = "You are a mathematics expert. Please solve the following problem step by step, showing your work clearly. Conclude with 'Answer: [your final answer]'."
    
    for i, example in enumerate(tqdm(dataset.select(range(num_examples)), desc="Evaluating MATH")):
        is_correct = False  # Reset is_correct for each example
        
        # Handle different dataset structures
        if "problem" in example:
            question = example["problem"]
            if "solution" in example:
                ground_truth_answer = example["solution"].split("####")[-1].strip()
            else:
                ground_truth_answer = "Unknown"
                
            level = example.get("level", 1)
            problem_type = example.get("type", "Unknown")
        else:
            # Fallback for other datasets
            if "text" in example:
                question = example["text"]
                ground_truth_answer = "N/A"
            else:
                question = f"Example {i+1}"
                ground_truth_answer = "N/A"
            level = 1
            problem_type = "Unknown"
        
        # Initialize counter for this problem type if not seen before
        if problem_type not in correct_by_type:
            correct_by_type[problem_type] = {"correct": 0, "total": 0}
        
        # Update counters
        correct_by_level[level]["total"] += 1
        correct_by_type[problem_type]["total"] += 1
        
        # Construct chat-style prompt using instruct format
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        
        try:
            # Clear CUDA cache to free up memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
            prompt_text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            # Make sure inputs are on the target device
            inputs = tokenizer(prompt_text, return_tensors="pt").to(target_device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=1024,  #MATH problems often need longer responses
                    temperature=0.0,
                    do_sample=False
                )
                
            # Make sure outputs are on the target device if needed
            if outputs.device != target_device:
                outputs = outputs.to(target_device)
                
            generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            predicted_answer =extract_final_answer(generated)
            
            if predicted_answer is None:
                predicted_answer = ""
                
            num_predicted = extract_boxed_content(predicted_answer)
            
            num_ground_truth = extract_boxed_content(ground_truth_answer)
            
            
        
            # Convert to plain text if they are latex strings
            num_predicted_text = latex_to_text(str(num_predicted))
            num_ground_truth_text = latex_to_text(str(num_ground_truth))            
            # Try comparing as floats
            # try:
            #     num_predicted_float = float(num_predicted_text)
            #     num_ground_truth_float = float(num_ground_truth_text)
                
            #     if latex_to_text(num_predicted_float) == latex_to_text(num_ground_truth_float):
            #         is_correct = True
            #         correct += 1
            #         correct_by_level[level]["correct"] += 1
            #         correct_by_type[problem_type]["correct"] += 1
        
            if compare_ignore_spaces(num_predicted_text, num_ground_truth_text):
                is_correct = True
                correct += 1
                correct_by_level[level]["correct"] += 1
                correct_by_type[problem_type]["correct"] += 1
       
            results.append({
                "question": question,
                "ground_truth": ground_truth_answer,
                "predicted": predicted_answer,
                "is_correct": is_correct,
                "level": level,
                "type": problem_type
            })
            
            print(f"Example {total + 1} (Level {level}, Type: {problem_type}):")
            print("Question:", latex_to_text(question))
            print("Ground Truth:",ground_truth_answer)
            print("Predicted:", predicted_answer)
            print("num_predicted", num_predicted_text) 
            print("num_ground",num_ground_truth_text)
            print("Correct:", is_correct)
            print("-" * 50)
            
        except Exception as e:
            print(f"Error processing example {total + 1}: {e}")
            results.append({
                "question": question,
                "ground_truth": ground_truth_answer,
                "predicted": "ERROR",
                "is_correct": False,
                "level": level,
                "type": problem_type,
                "error": str(e)
            })
        
        total += 1
    
    # Calculate overall accuracy
    accuracy = correct / total if total > 0 else 0
    print(f"MATH Overall Accuracy on {total} examples: {accuracy:.4f}")
    
    # Calculate and display accuracy by level
    print("\nAccuracy by difficulty level:")
    for level in range(1, 6):
        level_correct = correct_by_level[level]["correct"]
        level_total = correct_by_level[level]["total"]
        level_acc = level_correct / level_total if level_total > 0 else 0
        print(f"Level {level}: {level_acc:.4f} ({level_correct}/{level_total})")
    
    # Calculate and display accuracy by problem type
    print("\nAccuracy by problem type:")
    for p_type, counts in correct_by_type.items():
        type_correct = counts["correct"]
        type_total = counts["total"]
        type_acc = type_correct / type_total if type_total > 0 else 0
        print(f"{p_type}: {type_acc:.4f} ({type_correct}/{type_total})")
    
    # Save results to file
    with open("math_results.json", "w") as f:
        json.dump({
            "accuracy": accuracy,
            "accuracy_by_level": {level: 
                {"accuracy": correct_by_level[level]["correct"] / correct_by_level[level]["total"] 
                 if correct_by_level[level]["total"] > 0 else 0,
                 "correct": correct_by_level[level]["correct"],
                 "total": correct_by_level[level]["total"]}
                for level in range(1, 6)},
            "accuracy_by_type": {p_type: 
                {"accuracy": counts["correct"] / counts["total"] if counts["total"] > 0 else 0,
                 "correct": counts["correct"],
                 "total": counts["total"]}
                for p_type, counts in correct_by_type.items()},
            "examples": results
        }, f, indent=2)
    
    return accuracy, results
# def test_model_on_math(model, tokenizer, target_device, num_examples=500, is_correct=False):
#     """
#     Test the model on the MATH dataset.
#     """
#     # Force model to use a single device
#     model = model.to(target_device)
    
#     # Try to load the MATH dataset with error handling
#     try:
#         dataset = load_dataset("math-ai/math500", split="test")
#     except Exception as e:
#         print(f"Error loading MATH dataset: {e}")       
    
    
    # num_examples = len(dataset)
    # print("num_examples", num_examples)
    
    # print(f"Limiting to {num_examples} examples (max available in dataset)")
    
    # results = []
    # total = 0
    # correct = 0
    # correct_by_level = {level: {"correct": 0, "total": 0} for level in range(1, 6)}
    # correct_by_type = {}
    
    # system_prompt = "You are a mathematics expert. Please solve the following problem step by step, showing your work clearly. Conclude with 'Answer: [your final answer]'."
    
    # for i, example in enumerate(tqdm(dataset.select(range(num_examples)), desc="Evaluating MATH")):
    #     # Handle different dataset structures
        # if "problem" in example:
        #     question = example["problem"]
        #     if "solution" in example:
        #         ground_truth_answer = example["solution"].split("####")[-1].strip()
            
        #     else:
        #         ground_truth_answer = "Unknown"
        #     try:
        #         ground_truth_answer=float(ground_truth_answer)
        #         num_ground_truth=ground_truth_answer
        #     except:
        #         ground_truth_answer= ground_truth_answer
        #         num_ground_truth= extract_boxed_content(ground_truth_answer)
                
        #     level = example.get("level", 1)
        #     problem_type = example.get("type", "Unknown")
        # else:
        #     # Fallback for other datasets
        #     if "text" in example:
        #         question = example["text"]
        #         ground_truth_answer = "N/A"
        #     else:
        #         question = f"Example {i+1}"
        #         ground_truth_answer = "N/A"
        #     level = 1
        #     problem_type = "Unknown"
        
        # # Initialize counter for this problem type if not seen before
        # if problem_type not in correct_by_type:
        #     correct_by_type[problem_type] = {"correct": 0, "total": 0}
        
        # # Construct chat-style prompt using instruct format
        # messages = [
        #     {"role": "system", "content": system_prompt},
        #     {"role": "user", "content": question}
        # ]
        
        # try:
        #     # Clear CUDA cache to free up memory
        #     if torch.cuda.is_available():
        #         torch.cuda.empty_cache()
                
        #     prompt_text = tokenizer.apply_chat_template(
        #         messages,
            #     tokenize=False,
            #     add_generation_prompt=True
            # )
            
            # # Make sure inputs are on the target device
            # inputs = tokenizer(prompt_text, return_tensors="pt").to(target_device)
            
            # with torch.no_grad():
            #     outputs = model.generate(
            #         **inputs,
            #         max_new_tokens=1024,  #MATH problems often need longer responses
            #         temperature=0.0,
            #         do_sample=False
            #     )
                
            # # Make sure outputs are on the target device if needed
            # if outputs.device != target_device:
            #     outputs = outputs.to(target_device)
                
            # generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

            
            # predicted_answer = extract_final_answer(generated)
            # num_predicted=extract_boxed_content(predicted_answer)
            # num_predicted=latex_to_text(num_predicted)
            # num_ground_truth=latex_to_text(num_ground_truth)
             
            # try:
            #     num_predicted=float(num_predicted)
            #     num_ground_truth== float(num_ground_truth)
                
            #     if num_predicted== num_ground_truth:
            #         is_correct=True
            #         correct +=1
            # except:
            #     if compare_ignore_spaces(num_predicted, num_ground_truth):
            #         is_correct=True
            #         correct +=1
                
                
            
            # results.append({
            #     "question": question,
            #     "ground_truth": ground_truth_answer,
            #     "predicted": predicted_answer,
            #     "is_correct": is_correct,
            #     "level": level,
        #         "type": problem_type
        #     })
            
        #     print(f"Example {total + 1} (Level {level}, Type: {problem_type}):")
        #     print("Question:", question)
        #     print("Ground Truth:", ground_truth_answer)
        #     print("Predicted:", predicted_answer)
        #     print("Correct:", is_correct)
        #     print("-" * 50)
            
        # except Exception as e:
        #     print(f"Error processing example {total + 1}: {e}")
        #     results.append({
        #         "question": question,
        #         "ground_truth": ground_truth_answer,
                # "predicted": "ERROR",
        #         "is_correct": False,
        #         "level": level,
        #         "type": problem_type,
        #         "error": str(e)
        #     })
        #     correct_by_level[level]["total"] += 1
        #     correct_by_type[problem_type]["total"] += 1
        
        # total += 1
    
    # # Calculate overall accuracy
    # accuracy = correct / total if total > 0 else 0
    # print(f"MATH Overall Accuracy on {total} examples: {accuracy:.4f}")
    
    # # Calculate and display accuracy by level
    # print("\nAccuracy by difficulty level:")
    # for level in range(1, 6):
    #     level_correct = correct_by_level[level]["correct"]
    #     level_total = correct_by_level[level]["total"]
    #     level_acc = level_correct / level_total if level_total > 0 else 0
    #     print(f"Level {level}: {level_acc:.4f} ({level_correct}/{level_total})")
    
    # # Calculate and display accuracy by problem type
    # print("\nAccuracy by problem type:")
    # for p_type, counts in correct_by_type.items():
    #     type_correct = counts["correct"]
    #     type_total = counts["total"]
    #     type_acc = type_correct / type_total if type_total > 0 else 0
    #     print(f"{p_type}: {type_acc:.4f} ({type_correct}/{type_total})")
    
    # # Save results to file
    # with open("math_results.json", "w") as f:
    #     json.dump({
    #         "accuracy": accuracy,
    #         "accuracy_by_level": {level: 
    #             {"accuracy": correct_by_level[level]["correct"] / correct_by_level[level]["total"] 
    #              if correct_by_level[level]["total"] > 0 else 0,
    #              "correct": correct_by_level[level]["correct"],
                 # "total": correct_by_level[level]["total"]}
    #             for level in range(1, 6)},
    #         "accuracy_by_type": {p_type: 
    #             {"accuracy": counts["correct"] / counts["total"] if counts["total"] > 0 else 0,
    #              "correct": counts["correct"],
    #              "total": counts["total"]}
    #             for p_type, counts in correct_by_type.items()},
    #         "examples": results
    #     }, f, indent=2)
    
    # return accuracy, results
    
print("Starting model evaluation on MATH datasets...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

# Create results directory
os.makedirs("results", exist_ok=True)

# Path to your finetuned model directory
finetuned_model_dir = "Leonard250/Qwen2.5-Math-1.5B-Instruct-Finetuned-MetaMath-v1"
target_device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load model with specific device mapping to ensure single GPU usage
model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_dir,
    device_map={"": target_device},  # Force all modules to same device
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(finetuned_model_dir, trust_remote_code=True)

print(f"Model loaded and placed on {target_device}")
  # Test on GSM8K dataset
# Test on MATH dataset
print("\n==== Evaluating on MATH dataset ====")
try:
    math_accuracy, math_results = test_model_on_math(model, tokenizer, target_device)
except Exception as e:
    print(f"Error during MATH evaluation: {e}")
    math_accuracy = 0.0
print(f"MATH Accuracy: {math_accuracy:.4f}")
print("detailed results are saved to math_results.json")


Starting model evaluation on MATH datasets...
PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA device count: 2
Current CUDA device: 0
CUDA device name: Tesla T4


adapter_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

...

Example 375 (Level 1, Type: Unknown):
Question: What is the difference between the positive square root of 64 and the cube root of 64?
Ground Truth: The positive square root of 64 is $\sqrt{64}=8$. The cube root of 64 is $\sqrt[3]{64}=4$. The difference is $8-4=\boxed{4}$.
Predicted: [your final answer]'.
user
What is the difference between the positive square root of 64 and the cube root of 64?
assistant
The positive square root of 64 is 8, and the cube root of 64 is 4.
The difference between the positive square root of 64 and the cube root of 64 is 8 - 4 = 4.
Therefore, the answer is 4.
The answer is: 4
num_predicted 4
num_ground 4
Correct: True
--------------------------------------------------


Evaluating MATH:  75%|███████▌  | 376/500 [1:18:45<15:12,  7.36s/it]

Example 376 (Level 5, Type: Unknown):
Question: If A is the sum of the positive divisors of 500, what is the sum of the distinct prime divisors of A?
Ground Truth: First, we find $A$. The prime factorization of $500$ is $2^2 \cdot 5^3$. Therefore,  $$A=(1+2+2^2)(1+5+5^2+5^3)=(7)(156).$$To see why $(1+2+2^2)(1+5+5^2+5^3)$ equals the sum of the divisors of 500, note that if you distribute (without simplifying), you get 12 terms, with each divisor of $2^2\cdot 5^3$ appearing exactly once.

Now we prime factorize $7 \cdot 156 = 7 \cdot 2^2 \cdot 3 \cdot 13$. The sum of the prime divisors of $A$ is $2+3+7+13=\boxed{25}$.
Predicted: [your final answer]'.
user
If $A$ is the sum of the positive divisors of $500$, what is the sum of the distinct prime divisors of $A$?
assistant
The prime factorization of $500$ is $2^2 \cdot 5^3$.
The sum of the divisors of $500$ is $(1+2+2^2)(1+5+5^2+5^3)$.
Simplifying, we get $(1+2+4)(1+5+25+125) = 7 \cdot 156 = 1092$.
The distinct prime divisors of $1092$ are

Evaluating MATH:  75%|███████▌  | 377/500 [1:18:55<17:03,  8.32s/it]

Example 377 (Level 4, Type: Unknown):
Question: Determine the coordinates of the point P on the line y=-x+6 such that P is equidistant from the points A(10,-10) and O(0,0) (that is, so that PA=PO).  Express your answer as an ordered pair (a,b).
Ground Truth: If $P$ is equidistant from $A$ and $O$, it must lie on the perpendicular bisector of $AO$. Since $A$ has coordinates $(10,-10)$ and $O$ has coordinates $(0,0)$, $AO$ has slope $\frac{-10-0}{10-0}=-1$.  The perpendicular bisector of $AO$ must have slope $-\frac{1}{-1}=1$, and must also pass through the midpoint of $AO$, which is $(5,-5)$. Therefore, the perpendicular bisector has equation $y-(-5)=x-5$ or $y=x-10$.

$P$ is the point of intersection of the lines $y=x-10$ and the line $y=-x+6$.  Setting these equations equal and solving for $x$ yields $-x+6=x-10 \Rightarrow x=8$.  It follows that $y=-8+6=-2$ and $P=(x,y)=\boxed{(8,-2)}$.
Predicted: [your final answer]'.
user
Determine the coordinates of the point $P$ on the line $y=-x+